## Домашнее задание №2

### Собрать класс датасет с функциями из ноутбука по генерации датасета. Надо функции перенести в класс + добавить возможность при инициализации менять длину последовательности кадров.

In [81]:
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms

In [82]:
from torch.utils.data import DataLoader, Dataset

In [89]:
class Skeleton_Dataset(Dataset):
    def __init__(self, data_path, broken_files_path, chonk_len=45, transform=None):
        
        ##### список файлов с лейблами на каждый файл 
        self.chonk_len = chonk_len
        working_files_with_labels, action_classes = self.read_data(data_path, broken_files_path)
        
        self.data = self.generate_data(working_files_with_labels)
        self.transform = transform
        self.labels = self.data.iloc[:,-1]
        self.transform = transform


    def __len__(self):
        return len(self.data) 
    
    
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,75)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
        return (item, label) 
    
    def read_data(self, data_path, broken_files_path):
        labels = []
        files = []
        action_classes = {}
        counter = 0
        files_counter = {}

        with open(broken_files_path, 'r') as f:
            broken_files = f.read().split("\n")

        raw_files = os.listdir(data_path)
        num_frames = 0

        for filename in raw_files:
            if filename not in broken_files:
                action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                if action_class in training_classes and camera_id in training_cameras:  #and subject_id in training_subjects:
                    if action_class in action_classes:
                        if files_counter[action_class] < 120:
                            files.append([filename,action_classes[action_class]])
                            files_counter[action_class] = files_counter[action_class] + 1
                    else:
                        action_classes.update({action_class : counter})
                        files_counter.update({action_class : 1})
                        counter+=1
                        files.append([filename,action_classes[action_class]])
    #                     labels.append([action_class])
        print("action classes: ", action_classes)
        print("action files: ", files_counter)

        return files, action_classes

    def read_skeleton_filter(self, file):
        with open(file, 'r') as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []

                for m in range(frame_info['numBody']):
                    body_info = {}
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)

        return skeleton_sequence

    def read_xyz(self, file, max_body=1, num_joint=25):
        seq_info = read_skeleton_filter(file)
        data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < max_body and j < num_joint:
                        data[m, n, j, :] = [v['x'], v['y'], v['z']]

                    else:
                        pass

        return data
        
    def create_coords_blocks(self, test_file, chonk_len):   
        frame_counter = 0
        new_labels = []
        new_frames = []
        blocks = []

        test_frames = read_xyz(data_path + test_file[0])[0]
        label = test_file[1]
        slice_len = chonk_len * int(len(test_frames)/chonk_len)


        for index in range(len(test_frames[:slice_len])):
            frame_counter += 1
            new_frames.append(test_frames[index].flatten())
            if frame_counter == chonk_len:
                frame_counter = 0
                blocks.append(np.array(new_frames))
                new_labels = new_labels + [label]
                new_frames = []


        return blocks, new_labels
    
    def generate_data(self, working_files_with_labels):
        data = []
        labels = []
        ##########################################################################
        numbers = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0} #####
        ##################################################################
        for file in working_files_with_labels:
            frames_blocks, label = create_coords_blocks(file, self.chonk_len)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data = data + frames_blocks
                labels = labels + label
        data_np = np.asarray(data)
        labels_np = np.asarray(labels)

        data_sq = data_np.reshape(len(data_np), -1)
        test_data = pd.DataFrame(data_sq)
        test_labels = pd.DataFrame(labels_np)
        test_data['labels'] = test_labels
        return test_data


In [147]:
dataset = Skeleton_Dataset(data_path="nturgb+d_skeletons/", 
                           broken_files_path="NTU_RGBD_samples_with_missing_skeletons.txt",
                           chonk_len = 60
                           )

action classes:  {8: 0, 10: 1, 21: 2, 22: 3, 23: 4, 27: 5}
action files:  {8: 120, 10: 120, 21: 120, 22: 120, 23: 120, 27: 120}


In [148]:
dataset.data

,0,1,2,3,4,5,6,7,8,9,...,4491,4492,4493,4494,4495,4496,4497,4498,4499,labels
0,-0.080141,0.135073,3.691227,-0.058513,0.396866,3.606265,-0.037811,0.652906,3.509790,-0.066903,...,0.006913,-0.005347,3.876000,0.123996,-0.147997,3.730516,0.171769,-0.144677,3.733083,0
1,-0.035358,0.192430,3.681518,-0.021036,0.422171,3.606487,-0.007186,0.646429,3.520689,-0.042399,...,-0.179186,0.520803,3.414875,-0.279106,0.572582,3.399570,-0.238293,0.507033,3.378084,1
2,0.430001,0.024285,4.098490,0.424111,0.227365,4.005508,0.417157,0.427353,3.902021,0.395301,...,0.157654,-0.009465,3.886394,0.424538,-0.146789,3.790886,0.444941,-0.094041,3.782200,2
3,0.399282,-0.026338,4.099298,0.382165,0.177244,4.002089,0.359935,0.376536,3.893836,0.338453,...,0.384571,0.488159,3.896600,0.437795,0.698565,3.820046,0.451550,0.666552,3.821000,3
4,0.424387,0.040135,4.204897,0.398431,0.244353,4.116918,0.373611,0.444495,4.016130,0.355235,...,0.196250,-0.051198,3.967268,0.525538,0.701428,3.619313,0.506473,0.640511,3.593539,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,0.211163,-0.317639,2.506060,0.183202,-0.003583,2.478352,0.155062,0.302668,2.438098,0.210667,...,-0.105533,-0.558647,2.325600,0.309412,-0.506989,2.701166,0.262963,-0.499127,2.688874,0
638,0.217546,-0.325598,2.481236,0.191600,-0.012914,2.454174,0.163898,0.291428,2.412171,0.209757,...,0.460919,0.102599,2.313250,0.460319,0.048034,2.135425,0.495122,0.028175,2.139000,1
639,0.249771,-0.312151,2.500563,0.216183,0.001655,2.477956,0.182046,0.307419,2.441391,0.222758,...,0.342312,0.374436,2.098667,0.476951,0.488956,2.177114,0.474133,0.486067,2.164250,2
640,0.343596,-0.315473,2.519378,0.342929,-0.000757,2.502755,0.339000,0.305877,2.472029,0.380969,...,0.316832,0.706422,2.077177,0.839309,0.737348,2.304132,0.867375,0.680759,2.338250,3


In [149]:
dataset.data.labels.unique()

array([0, 1, 2, 3, 4, 5])

In [150]:
dataset.data.to_csv("skeletons_60_frames.csv", index = False)

In [151]:
aaa = pd.read_csv("skeletons_60_frames.csv")

In [152]:
aaa

,0,1,2,3,4,5,6,7,8,9,...,4491,4492,4493,4494,4495,4496,4497,4498,4499,labels
0,-0.080141,0.135073,3.691227,-0.058513,0.396866,3.606265,-0.037811,0.652906,3.509790,-0.066903,...,0.006913,-0.005347,3.876000,0.123996,-0.147997,3.730516,0.171769,-0.144677,3.733083,0
1,-0.035358,0.192430,3.681518,-0.021036,0.422171,3.606487,-0.007186,0.646429,3.520689,-0.042399,...,-0.179186,0.520803,3.414875,-0.279106,0.572582,3.399570,-0.238293,0.507033,3.378084,1
2,0.430001,0.024285,4.098490,0.424111,0.227365,4.005508,0.417157,0.427353,3.902021,0.395301,...,0.157654,-0.009465,3.886394,0.424538,-0.146789,3.790886,0.444941,-0.094041,3.782200,2
3,0.399282,-0.026338,4.099298,0.382165,0.177244,4.002089,0.359935,0.376536,3.893836,0.338453,...,0.384571,0.488159,3.896600,0.437795,0.698565,3.820046,0.451550,0.666552,3.821000,3
4,0.424387,0.040135,4.204897,0.398431,0.244353,4.116918,0.373611,0.444495,4.016130,0.355235,...,0.196250,-0.051198,3.967268,0.525538,0.701428,3.619313,0.506473,0.640511,3.593539,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,0.211163,-0.317639,2.506060,0.183202,-0.003583,2.478352,0.155062,0.302668,2.438098,0.210667,...,-0.105533,-0.558647,2.325600,0.309412,-0.506990,2.701166,0.262963,-0.499127,2.688874,0
638,0.217546,-0.325598,2.481236,0.191599,-0.012914,2.454174,0.163898,0.291428,2.412171,0.209757,...,0.460919,0.102599,2.313250,0.460319,0.048034,2.135425,0.495122,0.028175,2.139000,1
639,0.249771,-0.312151,2.500563,0.216183,0.001655,2.477956,0.182046,0.307419,2.441391,0.222758,...,0.342312,0.374436,2.098667,0.476951,0.488956,2.177114,0.474133,0.486067,2.164250,2
640,0.343596,-0.315473,2.519378,0.342929,-0.000757,2.502755,0.339000,0.305877,2.472029,0.380969,...,0.316832,0.706422,2.077177,0.839309,0.737348,2.304132,0.867375,0.680759,2.338250,3
